In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

data_path = 'data/'
df = pd.read_csv(data_path + 'taxi_data1.csv')
df

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,4.5,2011-10-21 23:54:10 UTC,-73.990580,40.761071,-73.981128,40.758634,2
1,12.0,2015-02-03 10:42:03 UTC,-73.988403,40.723431,-73.989647,40.741695,1
2,6.5,2014-03-16 18:58:58 UTC,-74.015785,40.715110,-74.012029,40.707888,2
3,6.5,2009-06-13 16:10:54 UTC,-73.977322,40.787275,-73.958030,40.778838,3
4,11.0,2014-06-12 03:25:56 UTC,-73.989683,40.729717,-73.982490,40.761887,3
...,...,...,...,...,...,...,...
4995,5.3,2011-12-23 19:07:00 UTC,-73.994150,40.761397,-73.986840,40.759202,1
4996,4.5,2009-08-26 18:19:41 UTC,-73.986767,40.729937,-73.977022,40.726482,1
4997,6.9,2010-10-25 06:05:21 UTC,-73.984612,40.732297,-73.995004,40.750059,1
4998,4.5,2010-02-16 16:57:00 UTC,-73.958460,40.773415,-73.953445,40.782598,1


In [2]:
train_Y = df['fare_amount']
df = df.drop(['fare_amount'], axis = 1)
df

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2011-10-21 23:54:10 UTC,-73.990580,40.761071,-73.981128,40.758634,2
1,2015-02-03 10:42:03 UTC,-73.988403,40.723431,-73.989647,40.741695,1
2,2014-03-16 18:58:58 UTC,-74.015785,40.715110,-74.012029,40.707888,2
3,2009-06-13 16:10:54 UTC,-73.977322,40.787275,-73.958030,40.778838,3
4,2014-06-12 03:25:56 UTC,-73.989683,40.729717,-73.982490,40.761887,3
...,...,...,...,...,...,...
4995,2011-12-23 19:07:00 UTC,-73.994150,40.761397,-73.986840,40.759202,1
4996,2009-08-26 18:19:41 UTC,-73.986767,40.729937,-73.977022,40.726482,1
4997,2010-10-25 06:05:21 UTC,-73.984612,40.732297,-73.995004,40.750059,1
4998,2010-02-16 16:57:00 UTC,-73.958460,40.773415,-73.953445,40.782598,1


In [3]:
df['pickup_datetime'] = df['pickup_datetime'].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S UTC'))
df['pickup_datetime']

0      2011-10-21 23:54:10
1      2015-02-03 10:42:03
2      2014-03-16 18:58:58
3      2009-06-13 16:10:54
4      2014-06-12 03:25:56
               ...        
4995   2011-12-23 19:07:00
4996   2009-08-26 18:19:41
4997   2010-10-25 06:05:21
4998   2010-02-16 16:57:00
4999   2009-02-18 17:01:05
Name: pickup_datetime, Length: 5000, dtype: datetime64[ns]

In [4]:
df['pickup_year'] = df['pickup_datetime'].apply(lambda x:datetime.datetime.strftime(x, '%Y')).astype('int64')
df['pickup_month'] = df['pickup_datetime'].apply(lambda x:datetime.datetime.strftime(x, '%m')).astype('int64')
df['pickup_day'] = df['pickup_datetime'].apply(lambda x:datetime.datetime.strftime(x, '%d')).astype('int64')
df['pickup_hour'] = df['pickup_datetime'].apply(lambda x:datetime.datetime.strftime(x, '%H')).astype('int64')
df['pickup_minute'] = df['pickup_datetime'].apply(lambda x:datetime.datetime.strftime(x, '%M')).astype('int64')
df['pickup_second'] = df['pickup_datetime'].apply(lambda x:datetime.datetime.strftime(x, '%S')).astype('int64')
df

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_second
0,2011-10-21 23:54:10,-73.990580,40.761071,-73.981128,40.758634,2,2011,10,21,23,54,10
1,2015-02-03 10:42:03,-73.988403,40.723431,-73.989647,40.741695,1,2015,2,3,10,42,3
2,2014-03-16 18:58:58,-74.015785,40.715110,-74.012029,40.707888,2,2014,3,16,18,58,58
3,2009-06-13 16:10:54,-73.977322,40.787275,-73.958030,40.778838,3,2009,6,13,16,10,54
4,2014-06-12 03:25:56,-73.989683,40.729717,-73.982490,40.761887,3,2014,6,12,3,25,56
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2011-12-23 19:07:00,-73.994150,40.761397,-73.986840,40.759202,1,2011,12,23,19,7,0
4996,2009-08-26 18:19:41,-73.986767,40.729937,-73.977022,40.726482,1,2009,8,26,18,19,41
4997,2010-10-25 06:05:21,-73.984612,40.732297,-73.995004,40.750059,1,2010,10,25,6,5,21
4998,2010-02-16 16:57:00,-73.958460,40.773415,-73.953445,40.782598,1,2010,2,16,16,57,0


In [ ]:
# 將結果使用線性迴歸 / 梯度提升樹分別看結果